In [ ]:
import numpy as np
import pandas as pd
import configparser
from googleapiclient.discovery import build

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config['youtube']['api_key']
youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
channel_name = pd.read_csv('data_channels.txt')
channel_name = channel_name.transpose()
channel_name.columns = ['Music','Gaming','Auto and Vehicles','Sports','Travel']
channel_name=channel_name.drop('Category',axis=0)

In [ ]:
def get_channel_stats(channel_name):
    search_response = youtube.search().list(
        q=channel_name,
        part='snippet',
        type='channel',
        maxResults=1  # We want the top result only
    ).execute()
    
    if 'items' not in search_response or len(search_response['items']) == 0:
        print(f"No channel found for: {channel_name}")
        return None
    
    channel_id = search_response['items'][0]['snippet']['channelId']
    
    channel_response = youtube.channels().list(
        id=channel_id,
        part='snippet,statistics'  # Fetch snippet (basic info) and statistics
    ).execute()
    
    if 'items' in channel_response and len(channel_response['items']) > 0:
        channel_data = channel_response['items'][0]
        
        channel_title = channel_data['snippet']['title']
        channel_description = channel_data['snippet']['description']
        view_count = channel_data['statistics'].get('viewCount', 'N/A')
        subscriber_count = channel_data['statistics'].get('subscriberCount', 'N/A')
        video_count = channel_data['statistics'].get('videoCount', 'N/A')

        print(f"Channel Title: {channel_title}")
        print(f"Channel ID: {channel_id}")
        print(f"Description: {channel_description}")
        print(f"View Count: {view_count}")
        print(f"Subscriber Count: {subscriber_count}")
        print(f"Video Count: {video_count}")

        return {
            'channel_title': channel_title,
            'channel_id': channel_id,
            'description': channel_description,
            'view_count': view_count,
            'subscriber_count': subscriber_count,
            'video_count': video_count
        }
    else:
        print(f"No statistics found for channel ID: {channel_id}")
        return None


In [ ]:
df_melted = channel_name.melt(var_name='Category', value_name='Channel')

In [ ]:
channel_data = df_melted['Channel'].apply(get_channel_stats)

In [ ]:
channel_df = pd.json_normalize(channel_data)
channel_df['Category'] = df_melted.Category